#HW1 - Clinical Trials Analytics in PySpark

### 1. Spark Installation


Download and install Spark with all its dependencies

In [60]:
#Install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# download spark3.4.4
#!wget -q https://apache.osuosl.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz

# unzip it
!tar xf spark-3.4.4-bin-hadoop3.tgz

# install findspark
!pip install -q findspark

It's necessary to add enviroment variables to make visible runtime Spark to linux enviroment. We could install different versions of spark and decide later which one we would use.

In [61]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

You import the library `findspark` that allow to find and automatically initialize Spark configuration without having to manually configure enviroment variable and other options

In [62]:
import findspark
findspark.init()

# Spark version verification on cluster
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

assert "3." in sc.version, "Verify that the cluster Spark's version is 3.x"

In [41]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### 2. Loading the Dataset on Spark


RDD is a data representation in Spark, but for simplicity of coding and design, it became necessary to introduce a new, more responsive data model.

Spark SQL came to live, it offers to the users the opportunity ot use datasets/dataframes. They are objects tablelike: each column has a name and a type, each row is a combination of column values

The SQL engine on Spark translates sql-like operations in RDDs and gives at the end a RDD with the results.

In [63]:
# Libraries for SQL Spark
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import split, explode, trim, count, sum, col, current_date, lower, regexp_replace
import time

spark = SparkSession(sc)
print(spark)

In [64]:
import zipfile
import requests

# Download the zip file from GitHub
url = "https://github.com/Salvr28/Big-Data-Homeworks/raw/main/Data/dimensions_clinicalTrials.zip"
zip_path = "dimensions_clinicalTrials.zip"

r = requests.get(url)
with open(zip_path, 'wb') as f:
  f.write(r.content)

# Extract the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(".")

# Read the CSV
extracted_files = os.listdir("./")
print("File estratti:", extracted_files)

csvPath = "./dimensions_clinicalTrials.csv"

File estratti: ['.config', 'spark-3.4.4-bin-hadoop3.tgz.2', 'spark-3.4.4-bin-hadoop3.tgz.1', 'spark-3.4.4-bin-hadoop3', 'drive', 'dimensions_clinicalTrials.csv', 'dimensions_clinicalTrials.zip', 'spark-3.4.4-bin-hadoop3.tgz', 'sample_data']


In [44]:
print(csvPath)

./dimensions_clinicalTrials.csv


In [65]:
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

# Percorso base per i notebooks Colab
base_path = '/content/drive/MyDrive/Colab Notebooks'

# Percorso completo per la cartella Big Data
big_data_path = os.path.join(base_path, 'Big Data')

# Verifica se la cartella Big Data esiste, altrimenti creala
if not os.path.exists(big_data_path):
    os.makedirs(big_data_path)
    print(f"Cartella creata: {big_data_path}")
else:
    print(f"La cartella esiste già: {big_data_path}")
    # Prova a stampare il contenuto se la cartella esiste
    try:
        print("Contenuto della cartella Big Data:")
        print(os.listdir(big_data_path))
    except:
        print("Impossibile leggere il contenuto della cartella")

# Percorso per Homework 1
homework1Path = os.path.join(big_data_path, "Homework 1")

# Verifica se la cartella Homework 1 esiste, altrimenti creala
if not os.path.exists(homework1Path):
    os.makedirs(homework1Path)
    print(f"Cartella creata: {homework1Path}")
else:
    print(f"La cartella esiste già: {homework1Path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
La cartella esiste già: /content/drive/MyDrive/Colab Notebooks/Big Data
Contenuto della cartella Big Data:
['Homework 1']
La cartella esiste già: /content/drive/MyDrive/Colab Notebooks/Big Data/Homework 1


### Dataset Object Creation

Create the Dataset Object with the spark Object `read`

In [66]:
# Indentifies types for each column (float, integer, string, etc)
# Gets columns names of the first csv's row
ctDS = spark.read \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("multiline", "true") \
  .option("quote", "\"") \
  .option("escape", "\"") \
  .csv(csvPath)

# Print the schema
ctDS.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Trial ID: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Brief title: string (nullable = true)
 |-- Acronym: string (nullable = true)
 |-- Abstract: string (nullable = true)
 |-- Start date: date (nullable = true)
 |-- Start Year: double (nullable = true)
 |-- End Date: date (nullable = true)
 |-- Completion Year: double (nullable = true)
 |-- Phase: string (nullable = true)
 |-- Study Type: string (nullable = true)
 |-- Study Design: string (nullable = true)
 |-- Conditions: string (nullable = true)
 |-- Recruitment Status: string (nullable = true)
 |-- Number of Participants: double (nullable = true)
 |-- Intervention: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Registry: string (nullable = true)
 |-- Investigators/Contacts: string (nullable = true)
 |-- Sponsors/Collaborators: string (nullable = true)
 |-- City of Sponsor/Collaborator: string (nullable = true

In [67]:
# Let's print some rows
ctDS.select("Study Design").show(20, truncate = False)

+----------------------------------------------------------------------------------------------------------------------------+
|Study Design                                                                                                                |
+----------------------------------------------------------------------------------------------------------------------------+
|Allocation: Randomized; Intervention Model: Parallel Assignment; Masking: None (Open Label); Primary Purpose: Treatment     |
|Allocation: N/A; Intervention Model: Single Group Assignment; Masking: None (Open Label); Primary Purpose: Supportive Care  |
|Allocation: Randomised Controlled Trial; Primary Purpose: Treatment                                                         |
|Allocation: Randomized; Intervention Model: Parallel Assignment; Masking: Double; Primary Purpose: Treatment                |
|Observational Model: Cohort                                                                                   

### 3. Preprocessing


At a first sight it could not be clear, but in this dataset there are a lot of duplicate rows. Clean the dataset with dropDuplicates()



In [68]:
# Dirty Dataset
count_dirty = ctDS.count()

# Cleaned Dataset
#ctDS = ctDS.fillna("NA_TEMP")
ctDS = ctDS.dropDuplicates(["Trial ID"])
#ctDS = ctDS.replace("NA_TEMP", None)
count_clean = ctDS.count()

print("Dirty: " + str(count_dirty))
print("Clean: " + str(count_clean))


Dirty: 15990
Clean: 8356


In [69]:
# Before the cleaning there were 7 rows with this "title"
ctDS.select("*").where(ctDS["Title"] == "Phase III Study on STem cElls Mobilization in Acute Myocardial Infarction").show(20,truncate=False)

+----+-----------+-------------------------------------------------------------------------+--------------------------------------------------------------------+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

There's still something wrong...Some columns have pseudo-structured data, like dictionaries and lists. In particular:

**Dictionaries**

*   Study Design

**Lists**

*   Conditions
*   Investigators/Contacts
*   Sponsors/Collaborators
*   City of Sponsor/Collaborator
*   State of Sponsor/Collaborator
*   Country of Sponsor/Collaborator
*   Fields of Research (ANZSRC 2020)
*   Funder Group
*   Funder Country
*   RCDC Categories
*   HRCS HC Categories
*   HRCS RAC Categories
*   Cancer Types
*   CSO Categories

**List of two elements**

*   Age (min ; max)

For better analytics, we have changed manually the type of these columns with a preprocessing script.



---



La prima trasformazione riguarda il **preprocessing delle pseudo-liste.**
1. Viene definita una lista chiamata `columnsList` che contiene i nomi di tutte le colonne da elaborare (contengono dati delimitati da ";")
2. Per ogni colonna nella lista, utilizzo il metodo `withColumn()` di PySpark per sostituire la colonna originale con una versione trasformata.
3. La trasformazione avviene tramite la funzione `expr()` che splitta la stringa in un array utilizzando il separatore `;` e applica una funzione di trasformazione che elimina gli spazi extra con `trim(x)` e converte eventuali stringhe vuote in valori `NULL` per ciascun elemento dell'array.




In [70]:
from pyspark.sql.functions import expr

# All columns that are pseudo-lists
columnsList = ["Conditions","Investigators/Contacts","Sponsors/Collaborators","City of Sponsor/Collaborator","State of Sponsor/Collaborator","Country of Sponsor/Collaborator","Fields of Research (ANZSRC 2020)","RCDC Categories","HRCS HC Categories","HRCS RAC Categories","Cancer Types","CSO Categories","Funder Group","Funder Country"]

# Cicle for all lists
for column in columnsList:
  ctDS = ctDS.withColumn(
      column,
      # Wiht expr function, we use sql like preprocessing
      expr(f"""
        transform(
            split(`{column}`, ';'),
            x -> CASE WHEN trim(x) = '' THEN NULL ELSE trim(x) END
        )
      """
    )
  )



La seconda trasformazione ci permette di trasformare la colonna "Study Design" in un dizionario, che in PySpark rappresentiamo con il Map Type (Key-Value).
1. La funzione UDF permette di creare e applicare una funzione creata dall'utente
2. Definiamo quindi la funzione `string_to_map` che crea un dizionario vuoto per memorizzare le coppie chiave-valore, splittando il testo in parti utilizzando il separatore `;`. Se è presente divide la parte in una coppia chiave-valore usando `:` come separatore. Aggiungiamo la coppia chiave-valore al dizionario `result` e restituisce il dizionario completo.
3. Creiamo quindi una UDF chiamata `string_to_map_udf` utilizzando la funzione `string_to_map`, specificando che il tipo da ritornare è `MapType(StringType(), StringType())`.
4. Applichiamo quindi la funzione UDF alla colonna "Study Design" del nostro DataFrame e sostituiamo la colonna originale.

Questo ci permetterà di fare operazioni più avanzate nelle nostre analytics.

In [71]:
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType

def string_to_map(text):
    if text is None:
        return None
    result = {}
    parts = text.split(';')
    for part in parts:
        if ':' in part:
            key, value = part.split(':', 1)
            result[key.strip()] = value.strip()
    return result

string_to_map_udf = udf(string_to_map, MapType(StringType(), StringType()))

ctDS = ctDS.withColumn("Study Design", string_to_map_udf(col("Study Design")))


La terza e ultima trasformazione riguarda la colonna "Age" che vogliamo trasformare in un Array Type gestendo i diversi formati di età standardizzandoli in anni.
1. Utilizziamo nuovamente UDF definendoci la funzione `age_preprocessing`.
2. Questa funzione divide il testo in due parti utilizzando il carattere "-" come separatore.
3. Elaboriamo prima il limite inferiore di età (prima parte del testo): se questo contiene "N/A", manteniamo il valore `None`, altrimenti cerchiamo un numero utilizzando una regex e lo convertiamo in *anni* in base all'unità di tempo trovata.
4. Facciamo la stessa elaborazione per il limite superiore di età (seconda parte del testo).
5. Registriamo la funzione UDF con tipo di ritorno `ArrayType(FloatType())` per gestire i valori decimali ed applichiamo la UDF alla colonna "Age".

Questo codice risolve i problemi nei formati inconsistenti (come 18 Years - 65 Years", "6 Months - 2 Years", "1 Hour - 24 Hours") ed i valori mancanti.

In [72]:
from pyspark.sql.types import ArrayType, IntegerType, FloatType
import re

def age_preprocessing(text):
  if text is None:
    return None

  # If the text contains 'N/A - N/A', return an array with None, None
  if text.strip() == "N/A - N/A":
    return [None, None]

  result = [None, None] #Inizalization with [None, None]
  parts = text.split("-")

  # Check whether we have exactly two parts
  if len(parts) == 2:
    #  We manage the minimum value (parts[0])
    if "N/A" in parts[0]:
      result[0] = None
    else:
      # We search the number
      number_match = re.search(r'\d+(\.\d+)?', parts[0])
      if number_match:
        value = float(number_match.group())

        # Convert to years
        if "month" in parts[0].lower() or "months" in parts[0].lower():
            result[0] = value / 12  # Convert months to years
        elif "hour" in parts[0].lower() or "hours" in parts[0].lower():
            result[0] = value / (24 * 365)  # Convert hours to years
        elif "day" in parts[0].lower() or "days" in parts[0].lower():
            result[0] = value / 365  # Convert days to years
        elif "week" in parts[0].lower() or "weeks" in parts[0].lower():
            result[0] = value / 52  # Convert weeks to years
        else:
            result[0] = value  # We assume that it is already in years

    # We manage the maximum value (parts[1])
    if "N/A" in parts[1]:
        result[1] = None
    else:
        number_match = re.search(r'\d+(\.\d+)?', parts[1])
        if number_match:
            value = float(number_match.group())

            if "month" in parts[1].lower() or "months" in parts[1].lower():
                result[1] = value / 12
            elif "hour" in parts[1].lower() or "hours" in parts[1].lower():
                result[1] = value / (24 * 365)
            elif "day" in parts[1].lower() or "days" in parts[1].lower():
                result[1] = value / 365
            elif "week" in parts[1].lower() or "weeks" in parts[1].lower():
                result[1] = value / 52
            else:
                result[1] = value

  return result

#Register the UDF
age_preprocessing_udf = udf(age_preprocessing, ArrayType(FloatType()))

# Apply the UDF to the "Age" column.
ctDS = ctDS.withColumn("Age", age_preprocessing_udf(col("Age")))

'''
def age_preprocessing(text):
  if text is None:
    return None

  result = []
  parts = text.split("-")
  if len(parts) == 2:
    for part in parts:
      matching = re.search(r'\d+(\.\d+)?',part)
      if matching:
        result.append(int(matching.group()))
      else:
        result.append(None)
  else:
    return None

  return result

age_preprocessing_udf = udf(age_preprocessing,ArrayType(IntegerType()))

ctDS = ctDS.withColumn("Age", age_preprocessing_udf(col("Age")))
'''


'\ndef age_preprocessing(text):\n  if text is None:\n    return None\n\n  result = []\n  parts = text.split("-")\n  if len(parts) == 2:\n    for part in parts:\n      matching = re.search(r\'\\d+(\\.\\d+)?\',part)\n      if matching:\n        result.append(int(matching.group()))\n      else:\n        result.append(None)\n  else:\n    return None\n\n  return result\n\nage_preprocessing_udf = udf(age_preprocessing,ArrayType(IntegerType()))\n\nctDS = ctDS.withColumn("Age", age_preprocessing_udf(col("Age")))\n'

In [73]:
#ctDS.select("*").show(100,truncate = True)
ctDS.select(col("Conditions")[0].alias("first_condition")).show(truncate = False)

+-------------------------+
|first_condition          |
+-------------------------+
|null                     |
|null                     |
|null                     |
|null                     |
|null                     |
|null                     |
|null                     |
|null                     |
|Chronic myeloid leukaemia|
|null                     |
|null                     |
|null                     |
|null                     |
|null                     |
|Epilepsy                 |
|Colorectal cancer NOS    |
|null                     |
|null                     |
|null                     |
|null                     |
+-------------------------+
only showing top 20 rows





---
Controlliamo la trasformazione sulla colonna Study Design.


In [74]:
total_rows = ctDS.count()
null_rows = ctDS.filter(ctDS["Study Design"].isNull()).count()

if total_rows == null_rows:
    print("La colonna è completamente null")
else:
    print(f"La colonna contiene {total_rows - null_rows} valori non-null")

ctDS.select("Study Design")  \
  .filter(ctDS["Study Design"].isNotNull()) \
  .show(truncate=False)

La colonna contiene 7352 valori non-null
+------------------------------------------------------------------------------------------------------------------------+
|Study Design                                                                                                            |
+------------------------------------------------------------------------------------------------------------------------+
|{Intervention Model -> Parallel, Masking -> No, Primary Purpose -> Treatment, Allocation -> Randomized Controlled Study}|
|{Intervention Model -> Parallel, Masking -> No, Primary Purpose -> Treatment, Allocation -> Randomized Controlled Study}|
|{Primary Purpose -> Basic Research/Physiological Study}                                                                 |
|{Primary Purpose -> Basic Research/Physiological Study}                                                                 |
|{Primary Purpose -> Treatment}                                                                   

In [75]:
ctDS.select("Study Design").where(ctDS["Trial ID"]=="NCT05817903").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------+
|Study Design                                                                                                                     |
+---------------------------------------------------------------------------------------------------------------------------------+
|{Intervention Model -> Parallel Assignment, Masking -> None (Open Label), Primary Purpose -> Treatment, Allocation -> Randomized}|
+---------------------------------------------------------------------------------------------------------------------------------+



---
Controlliamo trasformazione sulla colonna Age

In [76]:
ctDS.select("Age").where(ctDS["Age"][0].isNotNull() & (ctDS["Age"][0] < 1)).show(truncate=False)

+--------------------------+
|Age                       |
+--------------------------+
|[0.0, 21.0]               |
|[0.16666667, 6.0]         |
|[0.16666667, 6.0]         |
|[0.0, null]               |
|[0.002739726, 18.0]       |
|[0.5, 70.0]               |
|[0.002739726, 2.5]        |
|[0.5, 18.0]               |
|[0.5, 5.9166665]          |
|[0.15068494, 0.24383561]  |
|[0.083333336, 5.0]        |
|[0.15068494, 0.24383561]  |
|[0.083333336, 60.0]       |
|[0.42307693, 0.61538464]  |
|[0.083333336, 18.0]       |
|[0.083333336, 10.0]       |
|[0.6923077, null]         |
|[0.083333336, 2.0]        |
|[0.001369863, 0.002739726]|
|[0.002739726, 17.0]       |
+--------------------------+
only showing top 20 rows



### 4. Analytics

In [77]:
# Save all the results in csv format

# create the new repo path
resultsPath = os.path.join(homework1Path, "results")

# Check whether the results folder exists, otherwise create it
if not os.path.exists(resultsPath):
    os.makedirs(resultsPath)
    print(f"Cartella results creata: {resultsPath}")
else:
    print(f"La cartella results esiste già: {resultsPath}")

La cartella results esiste già: /content/drive/MyDrive/Colab Notebooks/Big Data/Homework 1/results


In [78]:
# Number of studies started per year
studiesPerYear = ctDS.select("Start Year") \
  .filter(ctDS["Start Year"].isNotNull()) \
  .groupBy(ctDS["Start Year"]) \
  .count() \
  .withColumnRenamed("count","NumStudies per Year") \
  .orderBy(col("NumStudies per Year").desc())

studiesPerYear.show(50,truncate = False)

# Saving result in a csv
studiesPerYearResult = studiesPerYear.toPandas()
studiesPerYearResult.to_csv(os.path.join(resultsPath,"studiesPerYear.csv"))


+----------+-------------------+
|Start Year|NumStudies per Year|
+----------+-------------------+
|2021.0    |722                |
|2020.0    |661                |
|2019.0    |640                |
|2018.0    |589                |
|2022.0    |585                |
|2017.0    |548                |
|2015.0    |457                |
|2016.0    |446                |
|2023.0    |399                |
|2014.0    |397                |
|2013.0    |370                |
|2012.0    |367                |
|2011.0    |325                |
|2010.0    |295                |
|2009.0    |287                |
|2008.0    |287                |
|2007.0    |226                |
|2006.0    |200                |
|2005.0    |130                |
|2004.0    |101                |
|2003.0    |56                 |
|2001.0    |45                 |
|2002.0    |40                 |
|2024.0    |38                 |
|2000.0    |35                 |
|1998.0    |19                 |
|1999.0    |18                 |
|1997.0   

In [79]:
# Average number of participants per study title

# Check if the title column is unique, maybe the question is refering title type
tot_rows = ctDS.count()

distinct_rows = ctDS.select("Title").distinct().count()

if tot_rows == distinct_rows:
  print("They're the same")
else:
  print("They're NOT the same")

# As shown they aren't the same, so we can group by title
print("Tot_rows: " + str(tot_rows))
print("Distinct_rows: " + str(distinct_rows))

They're NOT the same
Tot_rows: 8356
Distinct_rows: 8334


In [80]:
# Check which Title is duplicated, but the trials are different
duplicates = ctDS.groupBy("Title") \
  .count() \
  .filter(col("count") > 1)

duplicates.select("Title","count").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Title                                                                                                                                                                                                                                                                        |count|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|SAlute e LaVoro in Chirurgia Oncologica (SALVO)                                                                                                                      

In [81]:
from pyspark.sql.functions import avg

# Average number of participants per study title
averagePerTitle = ctDS.select("Title","Number of Participants") \
  .filter(ctDS["Title"].isNotNull()) \
  .groupBy("Title") \
  .agg(avg("Number of Participants").alias("Average per Title")) \
  .orderBy(col("Average per Title").desc())

averagePerTitle.show(20,truncate=False)

averagePerTitleResult = averagePerTitle.toPandas()
averagePerTitleResult.to_csv(os.path.join(resultsPath,"averagePerTitle.csv"))


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|Title                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [82]:
# Top 10 most frequent medical conditions
top10MedicalCondition = ctDS.select(explode(ctDS["Conditions"]).alias("Condition")) \
            .filter(col("Condition").isNotNull()) \
            .groupBy("Condition") \
            .count() \
            .withColumnRenamed("count","Count per Condition") \
            .orderBy(col("Count per Condition").desc()) \
            .limit(10)

top10MedicalCondition.show(truncate=False)

top10MedicalConditionResult = top10MedicalCondition.toPandas()
top10MedicalConditionResult.to_csv(os.path.join(resultsPath,"top10MedicalCondition.csv"))


+-----------------------+-------------------+
|Condition              |Count per Condition|
+-----------------------+-------------------+
|Breast Cancer          |157                |
|Multiple Myeloma       |83                 |
|Coronary Artery Disease|71                 |
|Heart Failure          |62                 |
|Ovarian Cancer         |60                 |
|Lung Cancer            |60                 |
|Colorectal Cancer      |56                 |
|Ulcerative Colitis     |52                 |
|Melanoma               |51                 |
|Prostate Cancer        |47                 |
+-----------------------+-------------------+



In [83]:
# Countries with the highest average number of participants per study
from pyspark.sql.functions import array_distinct
from pyspark.sql.functions import avg, row_number
from pyspark.sql.window import Window

# First query to retrieve Avg per study type/country
allAvgParticipants = ctDS.withColumn("Unique_Countries", array_distinct(col("Country of Sponsor/Collaborator"))) \
  .select("Study Type", explode(col("Unique_Countries")).alias("Country"),"Number of Participants") \
  .filter(col("Study Type").isNotNull() & col("Country").isNotNull() & col("Number of Participants").isNotNull()) \
  .groupBy("Study Type","Country") \
  .agg(avg("Number of Participants").alias("Average per Type/Country")) \
  .orderBy(col("Average per Type/Country").desc()) \

# Define a window to applicate the max to each study type
windowSpec = Window.partitionBy("Study Type").orderBy(col("Average per Type/Country").desc())

# Filter only max avg for each study type
maxCountryAvgPerStudyType = allAvgParticipants.withColumn("rank",row_number().over(windowSpec)) \
  .filter(col("rank")==1) \
  .select("Study Type","rank", "Country", "Average per Type/Country")

maxCountryAvgPerStudyType.show(100,truncate=False)

maxCountryAvgPerStudyTypeResult = maxCountryAvgPerStudyType.toPandas()
maxCountryAvgPerStudyTypeResult.to_csv(os.path.join(resultsPath,"maxCountryAvgPerStudyType"))

+-------------------+----+--------+------------------------+
|Study Type         |rank|Country |Average per Type/Country|
+-------------------+----+--------+------------------------+
|Active surveillance|1   |Italy   |115000.0                |
|CCT                |1   |Colombia|520.0                   |
|Interventional     |1   |Iceland |7924.888888888889       |
|Non-interventional |1   |Belgium |8122.5                  |
|Observational      |1   |Iceland |609000.0                |
|Other              |1   |Italy   |202.0                   |
+-------------------+----+--------+------------------------+



In [ ]:
# Range di anni (date) in cui hanno partecipato più persone agli studi (DIFFICILE: Bar race chart)
# Città che hanno trattato maggiormente una determinata condizione - FATTO
# Range di età che viene studiato/si sottopone per una determinata medical condition
# Nazione che sponsorizza di più clinical trial indirizzati al genere femminile - FATTO
# Nazione con il più alto numero di studi che comprende collaboratori minorenni - FATTO
# Stato che studia maggiormente un determinato tipo di cancro - FATTO
# Studi/Trial che attualmente sono terminati, con info anche su che tipo di condizioni trattano - FATTO
# Studi con Maggiore Visibilità Mediatica (Altmetric Score) - FATTO
# Condizioni che in media hanno più visibilità Mediatica (AlMetric Score) - FATTO
# Città/Nazione/Stato che ha sponsorizzato più studi - FATTO (CONSIDERANDO FUNDER COUNTRY)
# Investigator (studiosi?) che hanno fatto più trial
# Quante collaborazioni (trial) ha fatto la Federico II (AHC)
# Word Cloud su Brief Title/Title
# Sponsor/Collaborator (a coppie) che collaborano di più tra loro
# Percentuale di studi completati vs terminati (per patologia)




##### ATTENZIONE IN CONDITIONS C'è UN CAMPO MORE CONDITIONS CHE è UN DIZIONARIO #######
#### DA RIVEDERE QUERY CON SPONSOR/COLLABORATOR AL POSTO DI FUNDER GROUP ##########
#### MIGLIORARE PREPROCESSING CAMPO ETA' (ALCUNE INIZIANO PER MONTH E WEEKS) #######

In [ ]:
# Città che hanno trattato maggiormente una determinata condizione (Per condizione)

# Preprocessing, because select can't include more than one explode at time
cityConditionDS = ctDS.withColumn("City",explode(array_distinct(col("City of Sponsor/Collaborator")))) \
  .withColumn("Condition",explode(ctDS["Conditions"]))

# Defining the windowSpec
windowSpecCityCondition = Window.partitionBy("Condition").orderBy(col("NumStudies per City/Condition").desc())

# Here we could filter for a specific city, for instance where City == Bologna
cityConditionDS.select("City","Condition")\
  .groupBy("City","Condition") \
  .count() \
  .withColumnRenamed("count","NumStudies per City/Condition") \
  .withColumn("rank",row_number().over(windowSpecCityCondition)) \
  .filter((col("rank") == 1) & (col("City").isNotNull()) & (col("NumStudies per City/Condition") > 1)) \
  .select("City","Condition","NumStudies per City/Condition") \
  .show(20,truncate=False)


+----------------+---------------------------------------------------+-----------------------------+
|City            |Condition                                          |NumStudies per City/Condition|
+----------------+---------------------------------------------------+-----------------------------+
|Pavia           |AL Amyloidosis                                     |6                            |
|Milan           |ALS                                                |3                            |
|Naples          |Acid Maltase Deficiency                            |2                            |
|Orlando         |Acquired Immunodeficiency Syndrome                 |2                            |
|Yorkville       |Acral Lentiginous Melanoma                         |2                            |
|Brno            |Acute Ischemic Stroke                              |2                            |
|Rome            |Acute Myeloid Leukemia                             |29                   

In [ ]:
# Nation that sponsors the most clinical trials for women

ctDS.select(explode(array_distinct(ctDS["Country of Sponsor/Collaborator"])).alias("Nation")) \
  .where((ctDS["Gender"] == "Female") & (col("Nation").isNotNull())) \
  .groupBy("Nation") \
  .count() \
  .withColumnRenamed("count","NumFemaleStudies per Nation") \
  .orderBy(col("NumFemaleStudies per Nation").desc()) \
  .show(10,truncate=False)


+--------------+---------------------------+
|Nation        |NumFemaleStudies per Nation|
+--------------+---------------------------+
|Italy         |582                        |
|United States |176                        |
|Spain         |133                        |
|Belgium       |109                        |
|France        |106                        |
|Germany       |102                        |
|United Kingdom|102                        |
|Canada        |75                         |
|Poland        |71                         |
|Netherlands   |59                         |
+--------------+---------------------------+
only showing top 10 rows



In [ ]:
# Nazione con il più alto numero di studi che comprende collaboratori minorenni
ctDS.select(explode(array_distinct("Country of Sponsor/Collaborator")).alias("Country")) \
  .filter((col("Age").isNotNull()) & (col("Age")[0] < 18) & (col("Country").isNotNull())) \
  .groupBy("Country") \
  .count() \
  .withColumnRenamed("count","Count of Studies per Country") \
  .orderBy(col("Count of Studies per Country").desc()) \
  .show(20,truncate=False)

+--------------+----------------------------+
|Country       |Count of Studies per Country|
+--------------+----------------------------+
|Italy         |828                         |
|United States |475                         |
|Spain         |361                         |
|France        |356                         |
|United Kingdom|342                         |
|Germany       |339                         |
|Belgium       |265                         |
|Canada        |242                         |
|Poland        |220                         |
|Netherlands   |218                         |
|Australia     |190                         |
|Israel        |158                         |
|Austria       |133                         |
|Switzerland   |129                         |
|Hungary       |127                         |
|Czechia       |126                         |
|Sweden        |123                         |
|Japan         |115                         |
|Brazil        |106               

In [ ]:
#Gli stati che studiano di più un determinato tipo di cancro
# Preprocessing, because select can't include more than one explode at time
stateCancerDS = ctDS.withColumn("State",explode(array_distinct(col("State of Sponsor/Collaborator")))) \
  .withColumn("Cancer Type",explode(ctDS["Cancer Types"]))

# Defining the windowSpec
windowSpecStateCancer = Window.partitionBy("Cancer Type").orderBy(col("NumStudies per State/Cancer").desc())

# Here we could filter for a specific city, for instance where City == Bologna
stateCancerDS.select("State","Cancer Type")\
  .groupBy("State","Cancer Type") \
  .count() \
  .withColumnRenamed("count","NumStudies per State/Cancer") \
  .withColumn("rank",row_number().over(windowSpecStateCancer)) \
  .filter((col("rank") == 1) & (col("State").isNotNull())) \
  .show(20,truncate=False)

+-----+-----------+---------------------------+----+
|State|Cancer Type|NumStudies per State/Cancer|rank|
+-----+-----------+---------------------------+----+
+-----+-----------+---------------------------+----+



In [ ]:
# Per Stati non si può fare perchè gli Stati con rank = 1 sono null, quindi facciamo per città
stateCancerDS.select("State","Cancer Type")\
  .groupBy("State","Cancer Type") \
  .count() \
  .withColumnRenamed("count","NumStudies per State/Cancer") \
  .withColumn("rank",row_number().over(windowSpecStateCancer)) \
  .orderBy(col("rank")) \
  .show(100,truncate=False)


+---------------+----------------------------------------------------------+---------------------------+----+
|State          |Cancer Type                                               |NumStudies per State/Cancer|rank|
+---------------+----------------------------------------------------------+---------------------------+----+
|null           |Kaposi's Sarcoma                                          |4                          |1   |
|null           |Anal Cancer                                               |4                          |1   |
|null           |Bladder Cancer                                            |58                         |1   |
|null           |Blood Cancer                                              |46                         |1   |
|null           |Bone Cancer, Osteosarcoma / Malignant Fibrous Histiocytoma|34                         |1   |
|null           |Breast Cancer                                             |340                        |1   |
|null     

In [ ]:
#Le che studiano di più un determinato tipo di cancro
# Preprocessing, because select can't include more than one explode at time
cityCancerDS = ctDS.withColumn("City",explode(array_distinct(col("City of Sponsor/Collaborator")))) \
  .withColumn("Cancer Type",explode(ctDS["Cancer Types"]))

# Defining the windowSpec
windowSpecCityCancer = Window.partitionBy("Cancer Type").orderBy(col("NumStudies per City/Cancer").desc())

# Here we could filter for a specific city, for instance where City == Bologna
cityCancerDS.select("City","Cancer Type")\
  .groupBy("City","Cancer Type") \
  .count() \
  .withColumnRenamed("count","NumStudies per City/Cancer") \
  .withColumn("rank",row_number().over(windowSpecCityCancer)) \
  .filter((col("rank") == 1) & (col("City").isNotNull())) \
  .orderBy(col("NumStudies per City/Cancer").desc()) \
  .show(20,truncate=False)

+-------+----------------------------------------------------------+--------------------------+----+
|City   |Cancer Type                                               |NumStudies per City/Cancer|rank|
+-------+----------------------------------------------------------+--------------------------+----+
|Milan  |Lung Cancer                                               |255                       |1   |
|Rome   |Leukemia / Leukaemia                                      |218                       |1   |
|Milan  |Non-Hodgkin's Lymphoma                                    |216                       |1   |
|Milan  |Breast Cancer                                             |190                       |1   |
|Milan  |Colon and Rectal Cancer                                   |136                       |1   |
|Milan  |Not Site-Specific Cancer                                  |101                       |1   |
|Milan  |Liver Cancer                                              |89                     

In [ ]:
# Studi/Trial che attualmente sono terminati, con info anche su che tipo di condizioni trattano
from pyspark.sql.functions import to_date, current_date

ctDS.withColumn("End Date",to_date("End Date","yyyy-MM-dd")) \
  .select("Trial ID","End Date",explode(col("Conditions")).alias("Condition")) \
  .where(col("End Date") < current_date() & (col("Condition").isNotNull())) \
  .orderBy(col("End Date").desc()) \
  .show(truncate=False)

+--------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Trial ID      |End Date  |Condition                                                                                                                                                                     |
+--------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|NCT03370172   |2025-04-24|Hemophilia A                                                                                                                                                                  |
|NCT05405114   |2025-04-16|Sickle Cell Disease                                                                                                                                              

In [ ]:
# Studi con Maggiore Visibilità Mediatica (Altmetric Score)
ctDS.select("Trial ID","Altmetric Attention Score",explode(col("Conditions")).alias("Condition")) \
  .where(col("Altmetric Attention Score").isNotNull() & col("Condition").isNotNull()) \
  .orderBy(col("Altmetric Attention Score").desc()) \
  .show(truncate=False)

+-----------+-------------------------+-----------------------------------------------+
|Trial ID   |Altmetric Attention Score|Condition                                      |
+-----------+-------------------------+-----------------------------------------------+
|NCT04575597|1703.0                   |Coronavirus Disease (COVID-19)                 |
|NCT02819635|1283.0                   |Ulcerative Colitis (UC)                        |
|NCT03104400|1247.0                   |Psoriatic Arthritis                            |
|NCT03105128|1129.0                   |Crohn's Disease                                |
|NCT04292899|1081.0                   |COVID-19                                       |
|NCT03398148|1061.0                   |Ulcerative Colitis (UC)                        |
|NCT03398135|1030.0                   |Ulcerative Colitis (UC)                        |
|NCT03569293|1010.0                   |Atopic Dermatitis                              |
|NCT03671148|962.0              

In [ ]:
# Condizioni che in media hanno più visibilità Mediatica (AlMetric Attention Score)
ctDS.select(explode(col("Conditions")).alias("Condition"),"Altmetric Attention Score") \
  .filter(col("Condition").isNotNull()) \
  .groupBy("Condition") \
  .agg(avg("Altmetric Attention Score").alias("Average Attention Score")) \
  .orderBy(col("Average Attention Score").desc()) \
  .show(truncate=False)

+-----------------------------------------------+-----------------------+
|Condition                                      |Average Attention Score|
+-----------------------------------------------+-----------------------+
|Coronavirus Disease (COVID-19)                 |1703.0                 |
|Psoriatic Arthritis (PsA)                      |962.0                  |
|Untreated AML                                  |834.0                  |
|Newly Diagnosed Acute Myeloid Leukemia (AML)   |834.0                  |
|AML Arising From Myelodysplastic Syndrome (MDS)|834.0                  |
|KRAS p, G12c Mutated /Advanced Metastatic NSCLC|788.0                  |
|Central Nervous System Neoplasms               |784.0                  |
|Breast Diseases                                |784.0                  |
|Brain Neoplasms                                |784.0                  |
|Retinal Vein Occlusion                         |774.0                  |
|Glioblastoma Multiforme, Adult       

In [87]:
#Paesi degli enti finanziatori che hanno finanziato maggiormente studi
topFunderCountryDS = ctDS.filter(col("Funder Country").isNotNull()) \
                .withColumn("Funder Country", explode(array_distinct("Funder Country"))) \
                .groupBy("Funder Country") \
                .count() \
                .withColumnRenamed("count", "Number of Studies") \
                .orderBy(col("Number of Studies").desc()) \
                .select("Funder Country", "Number of Studies") \
                .show(20, truncate=False)

+--------------+-----------------+
|Funder Country|Number of Studies|
+--------------+-----------------+
|Italy         |1592             |
|United States |1529             |
|Germany       |277              |
|Japan         |191              |
|Belgium       |183              |
|United Kingdom|182              |
|Switzerland   |119              |
|Netherlands   |83               |
|France        |79               |
|Spain         |47               |
|Canada        |39               |
|Denmark       |36               |
|Sweden        |32               |
|Australia     |26               |
|Austria       |20               |
|Norway        |14               |
|Portugal      |12               |
|Ireland       |11               |
|Finland       |10               |
|Brazil        |8                |
+--------------+-----------------+
only showing top 20 rows



In [97]:
#Studiosi che hanno partecipato a più studi
topInvestigatorsDs = ctDS.filter(col("Investigators/Contacts").isNotNull()) \
                .withColumn("Investigator", explode("Investigators/Contacts")) \
                .withColumn("Investigator", trim(lower(col("Investigator")))) \
                .groupBy("Investigator") \
                .count() \
                .withColumnRenamed("count", "Number of Studies") \
                .orderBy(col("Number of Studies").desc()) \
                .select("Investigator", "Number of Studies") \
                .show(20, truncate=False)

+--------------------+-----------------+
|Investigator        |Number of Studies|
+--------------------+-----------------+
|abbvie inc          |89               |
|francesco perrone   |52               |
|boehringer ingelheim|52               |
|sandro pignata      |27               |
|antonio carroccio   |25               |
|nicola cascavilla   |25               |
|ciro gallo          |24               |
|filippo de braud    |24               |
|alessandro rambaldi |23               |
|lisa licitra        |22               |
|nicola di renzo     |21               |
|davide chiumello    |21               |
|michele spina       |20               |
|francesco zaja      |19               |
|renato bassan       |18               |
|bryan a. faller     |18               |
|luca arcaini        |18               |
|cesare gridelli     |18               |
|fabrizio pane       |18               |
|site                |17               |
+--------------------+-----------------+
only showing top

In [98]:
investigatorConditionsDS = ctDS.filter(col("Investigators/Contacts").isNotNull() & col("Conditions").isNotNull()) \
                    .withColumn("Investigator", explode("Investigators/Contacts")) \
                    .withColumn("Investigator", trim(lower(col("Investigator")))) \
                    .withColumn("Condition", explode("Conditions")) \
                    .withColumn("Condition", trim(lower(col("Condition"))))

windowSpecNumStudies = Window.partitionBy("Investigator").orderBy(col("Number of Studies").desc())

topConditionPerInvestigator = investigatorConditionsDS.groupBy("Investigator", "Condition") \
                        .count() \
                        .withColumnRenamed("count", "Number of Studies") \
                        .withColumn("rank", row_number().over(windowSpecNumStudies)) \
                        .filter(col("rank") == 1) \
                        .select("Investigator", "Number of Studies", "Condition") \
                        .orderBy(col("Number of Studies").desc()) \
                        .show(20, truncate=False)

+-----------------------+-----------------+---------------------------------------+
|Investigator           |Number of Studies|Condition                              |
+-----------------------+-----------------+---------------------------------------+
|sandro pignata         |14               |ovarian cancer                         |
|antonio carroccio      |11               |non-celiac wheat sensitivity           |
|abbvie inc             |9                |crohn's disease                        |
|francesco perrone      |9                |ovarian cancer                         |
|luca elli              |9                |celiac disease                         |
|alessio g morganti     |8                |radiotherapy                           |
|michelino de laurentiis|8                |metastatic breast cancer               |
|fabio puglisi          |7                |metastatic breast cancer               |
|lorenzo livi           |6                |breast cancer                    

In [99]:
topInvestigatorsDs = ctDS.filter(col("Investigators/Contacts").isNotNull()) \
                .withColumn("Investigator", explode("Investigators/Contacts")) \
                .withColumn("Investigator", trim(lower(col("Investigator")))) \
                .groupBy("Investigator") \
                .count() \
                .withColumnRenamed("count", "Number of Studies")

conditionCountDs = ctDS.filter(col("Investigators/Contacts").isNotNull() & col("Conditions").isNotNull()) \
              .withColumn("Investigator", explode("Investigators/Contacts")) \
              .withColumn("Investigator", trim(lower(col("Investigator")))) \
              .withColumn("Condition", explode("Conditions")) \
              .withColumn("Condition", trim(lower(col("Condition")))) \
              .groupBy("Investigator", "Condition") \
              .count() \
              .withColumnRenamed("count", "ConditionCount")

# Applica una finestra per ottenere la condizione top per ciascun Investigatore
windowSpecTopCondition = Window.partitionBy("Investigator").orderBy(col("ConditionCount").desc())

topConditionPerInvestigator = conditionCountDs.withColumn("rank", row_number().over(windowSpecTopCondition)) \
              .filter(col("rank")==1) \
              .select("Investigator", "Condition") \
              .withColumnRenamed("Condition", "Top Condition")

#Join
topInvestigatorWithCondition = topInvestigatorsDs.join(topConditionPerInvestigator, on="Investigator", how="left") \
              .orderBy(col("Number of Studies").desc()) \
              .show(20,truncate=False)


+--------------------+-----------------+-----------------------------------+
|Investigator        |Number of Studies|Top Condition                      |
+--------------------+-----------------+-----------------------------------+
|abbvie inc          |89               |crohn's disease                    |
|francesco perrone   |52               |ovarian cancer                     |
|boehringer ingelheim|52               |carcinoma, non-small-cell lung     |
|sandro pignata      |27               |ovarian cancer                     |
|antonio carroccio   |25               |non-celiac wheat sensitivity       |
|nicola cascavilla   |25               |acute myeloid leukemia             |
|ciro gallo          |24               |ovarian cancer                     |
|filippo de braud    |24               |breast cancer                      |
|alessandro rambaldi |23               |chronic myeloid leukemia           |
|lisa licitra        |22               |head and neck cancer               |